In [1]:
import pandas as pd
import numpy as np
import re

import plotly.express as px

In [2]:
df_raw = pd.read_csv('../Data/drug_data.csv', index_col=0)
df_raw.head()

,CASEID,DRUGID_1,CATID_1_1,CATID_2_1,CATID_3_1,ROUTE_1,TOXTEST_1,sdled_1_1,sdled_2_1,sdled_3_1,...,QUARTER,DAYPART,NUMSUBS,CASETYPE,DISPOSITION,ALLABUSE,METRO,AGECAT,SEX,RACE
0,1,865,105,110,2005,1,2,1,2.0,-7.00,...,4,1,2,8,6,1,2,4,1,2
1,2,2077,81,82,283,-8,2,17,67.0,67.01,...,3,4,1,4,7,0,2,11,1,3
2,3,2313,1,12,-7,-8,2,17,64.0,64.99,...,4,3,1,4,1,0,7,11,2,2
3,4,234,358,99,215,2,2,17,73.0,73.01,...,2,2,1,4,7,0,10,2,1,3
4,5,865,105,110,2005,1,2,1,2.0,-7.00,...,3,4,3,8,8,1,1,6,1,3


In [4]:
df_raw[df_raw.eq(1255).any(axis=1)]

,CASEID,DRUGID_1,CATID_1_1,CATID_2_1,CATID_3_1,ROUTE_1,TOXTEST_1,sdled_1_1,sdled_2_1,sdled_3_1,...,QUARTER,DAYPART,NUMSUBS,CASETYPE,DISPOSITION,ALLABUSE,METRO,AGECAT,SEX,RACE
0,1,865,105,110,2005,1,2,1,2.0,-7.0,...,4,1,2,8,6,1,2,4,1,2
6,7,1255,105,114,2032,4,2,1,2.5,5.0,...,1,1,1,8,1,1,2,5,2,3
11,12,1254,105,114,2032,-8,1,1,2.5,3.0,...,4,4,2,8,2,1,1,9,1,-8
20,21,505,57,71,-7,-8,1,1,2.5,5.5,...,3,3,3,8,1,1,13,5,2,1
32,34,1255,105,114,2032,4,2,1,2.5,5.0,...,4,4,3,8,6,1,14,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218895,229155,1254,105,114,2032,-8,2,1,2.5,3.0,...,3,3,2,8,6,1,2,6,1,2
218908,229169,865,105,110,2005,1,2,1,2.0,-7.0,...,2,3,2,8,1,1,4,9,1,-8
218920,229182,1255,105,114,2032,4,2,1,2.5,5.0,...,1,4,1,8,8,1,3,6,1,3
218925,229187,865,105,110,2005,-8,2,1,2.0,-7.0,...,1,2,2,8,6,1,9,9,1,1


Find sdleds to drop

In [5]:
sdled_cols = [column for column in df_raw.columns if column.startswith('sdled')]
sdled_cols

['sdled_1_1',
 'sdled_2_1',
 'sdled_3_1',
 'sdled_4_1',
 'sdled_5_1',
 'sdled_6_1',
 'sdled_1_2',
 'sdled_2_2',
 'sdled_3_2',
 'sdled_4_2',
 'sdled_5_2',
 'sdled_6_2',
 'sdled_1_3',
 'sdled_2_3',
 'sdled_3_3',
 'sdled_4_3',
 'sdled_5_3',
 'sdled_6_3']

In [6]:
def pct_na(col):
    freq = col.value_counts()
    if -7 in freq.index:
        return (freq[-7] / df_raw.shape[0]) * 100
    else:
        return 0

df_sdled_na_pct = df_raw[sdled_cols].apply(lambda x: pct_na(x), axis=0)
df_sdled_na_pct

sdled_1_1     0.000000
sdled_2_1     0.000000
sdled_3_1    22.422471
sdled_4_1    48.588262
sdled_5_1    74.691025
sdled_6_1    92.030601
sdled_1_2    67.200274
sdled_2_2    67.200274
sdled_3_2    69.005252
sdled_4_2    79.985842
sdled_5_2    88.550811
sdled_6_2    96.789221
sdled_1_3    92.133821
sdled_2_3    92.133821
sdled_3_3    92.504681
sdled_4_3    94.554921
sdled_5_3    96.680977
sdled_6_3    99.153231
dtype: float64

In [7]:
sdleds_todrop = [col for col in sdled_cols if re.match(r'^sdled_[5-6]', col)]

In [8]:
df_new = df_raw.drop(columns = sdleds_todrop)
df_new.columns

Index(['CASEID', 'DRUGID_1', 'CATID_1_1', 'CATID_2_1', 'CATID_3_1', 'ROUTE_1',
       'TOXTEST_1', 'sdled_1_1', 'sdled_2_1', 'sdled_3_1', 'sdled_4_1',
       'DRUGID_2', 'CATID_1_2', 'CATID_2_2', 'CATID_3_2', 'ROUTE_2',
       'TOXTEST_2', 'sdled_1_2', 'sdled_2_2', 'sdled_3_2', 'sdled_4_2',
       'DRUGID_3', 'CATID_1_3', 'CATID_2_3', 'CATID_3_3', 'ROUTE_3',
       'TOXTEST_3', 'sdled_1_3', 'sdled_2_3', 'sdled_3_3', 'sdled_4_3',
       'ALCOHOL', 'NONALCILL', 'PHARMA', 'NONMEDPHARMA', 'CASEWGT', 'YEAR',
       'QUARTER', 'DAYPART', 'NUMSUBS', 'CASETYPE', 'DISPOSITION', 'ALLABUSE',
       'METRO', 'AGECAT', 'SEX', 'RACE'],
      dtype='object')

Find catid's to drop

In [37]:
catid_cols = [column for column in df_raw.columns if column.startswith('CATID')]
catid_cols

['CATID_1_1',
 'CATID_2_1',
 'CATID_3_1',
 'CATID_1_2',
 'CATID_2_2',
 'CATID_3_2',
 'CATID_1_3',
 'CATID_2_3',
 'CATID_3_3']

In [41]:
def pct_na(col):
    freq = col.value_counts()
    if -7 in freq.index:
        return (freq[-7] / df_raw.shape[0]) * 100
    else:
        return 0

df_catid_na_pct = df_raw[catid_cols].apply(lambda x: pct_na(x), axis=0)
df_catid_na_pct

CATID_1_1     0.000000
CATID_2_1     3.730532
CATID_3_1    21.995433
CATID_1_2    67.200274
CATID_2_2    68.936744
CATID_3_2    72.807034
CATID_1_3    92.133821
CATID_2_3    92.481845
CATID_3_3    93.461064
dtype: float64

Value maps for Top drugs in DRUGID 1, 2, and 3

In [32]:
top_drugids = {
    'DRUGID_1': [1255, 1254, 1253, 865, 2420, 21],
    'DRUGID_2': [1255, 1254],
    'DRUGID_3': [1255, 1254]
}

In [33]:
def find_related_list(drugid_col: str, drugids: list):
    drugid_map = {}
    for drugid in drugids:
        df_1 = df_new[df_new[drugid_col]==drugid]
        related_cols = [col for col in df_new.columns if (
            col.startswith(('CATID', 'sdled')) and col.endswith('_1'))]
        col_dict = {}
        for col in related_cols:
            col_dict[col] = df_1[col].unique()
        drugid_map[drugid] = col_dict
    
    return drugid_map

In [37]:
final_dict = {}
for did in top_drugids.keys():
    map_name = f'{did}_map'
    final_dict[map_name] = find_related_list(drugid_col=did, drugids=top_drugids[did])

In [38]:
final_dict

{'DRUGID_1_map': {1255: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([5.]),
   'sdled_4_1': array([5.01])},
  1254: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([3.]),
   'sdled_4_1': array([-7.])},
  1253: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([4.]),
   'sdled_4_1': array([-7.])},
  865: {'CATID_1_1': array([105]),
   'CATID_2_1': array([110]),
   'CATID_3_1': array([2005]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.]),
   'sdled_3_1': array([-7.]),
   'sdled_4_1': array([-7.])},
  2420: {'CATID_1_1': array([2006]),
   'CATID_2_1': array([-7]),
   'CATID_3_1': array([-7]),
   'sdled_1_1': array([17]),


Get demographic dataset


In [2]:
df_raw = pd.read_csv('../Data/drug_data.csv', index_col=0)
demographic_df = df_raw[['CASEID', 'METRO', 'AGECAT', 'SEX', 'RACE','DISPOSITION', 'ALLABUSE', 'CASETYPE', 'ALCOHOL', 'NONALCILL', 'PHARMA', 'NONMEDPHARMA']]
demographic_df.replace({-7:0,-8:0,-9:0},inplace=True)
demographic_df['DISPOSITION'].replace({96:11}, inplace=True)
demographic_df.head()

/tmp/ipykernel_5875/2111754696.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_df.replace({-7:0,-8:0,-9:0},inplace=True)
/tmp/ipykernel_5875/2111754696.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_df['DISPOSITION'].replace({96:11}, inplace=True)


,CASEID,METRO,AGECAT,SEX,RACE,DISPOSITION,ALLABUSE,CASETYPE,ALCOHOL,NONALCILL,PHARMA,NONMEDPHARMA
0,1,2,4,1,2,6,1,8,1,1,0,0
1,2,2,11,1,3,7,0,4,0,0,1,0
2,3,7,11,2,2,1,0,4,0,0,1,0
3,4,10,2,1,3,7,0,4,0,0,1,0
4,5,1,6,1,3,8,1,8,1,1,0,0


In [3]:
demographic_df.to_csv('../Data/demographic_data.csv')

In [4]:
df_grouped = demographic_df.groupby(['AGECAT', 'CASETYPE']).agg(
    count_col=pd.NamedAgg(column='CASETYPE', aggfunc="count")
        )

In [39]:
agecat_map = {0:'NA', 1:'<= 5', 2:'6-11',3:'12-17', 4:'18-20', 5:'21-24', 6:'25-29', 7:'30-34', 8:'35-44', 9:'45-54', 10:'55-64',11:'>= 65'}
sex_map = {0:'NA', 1:'Male', 2:'Female'}
race_map = {0:'NA', 1:'White', 2:'Black/African American',3:'Any Hispanic/Latino', 4:'All Other'}
casetype_map = {0: 'NA', 1:'Suicide Attempt', 2:'Seeking Detox',3:'Alcohol Only(Age<21)', 4:'Adverse Reaction', 5:'Overmedication', 6:'Malicious Poisoning', 7:'Accidental Injestion', 8:'Other'}
metro_map = {0:'NA', 1:'MA-NH', 2:'NY-NJ-PA',3:'IL-IN-WI', 4:'MI', 5:'MN-WI', 6:'FL', 7:'FL', 8:'TX', 9:'CO', 10:'AZ',11:'CA', 12:'CA', 13:'WA', 14:'Other'}
alcohol_map = {0:'Alcohol not mentioned', 1: 'Alcohol mentioned'}
nonalcill_map = {0:'Episode doesn\'t involve illicit drugs', 1: 'Episode involves illicit drugs'}
pharma_map = {0:'Episode doesn\'t involve pharmaceuticals', 1: 'Episode involves pharmaceuticals'}
nonmedpharma_map = {0:'Episode doesn\'t involve non-medical use of pharmaceuticals', 1: 'Episode involves non-medical use of pharmaceuticals'}
# index_0 = df_grouped.index.get_level_values(0).map(agecat_map)
# index_1 = df_grouped.index.get_level_values(1).map(casetype_map)

In [7]:
df_grouped_mapped = df_grouped.set_index([index_0, index_1])
df_grouped_mapped['count_col'].groupby('AGECAT', group_keys=False).nlargest(3)

In [ ]:
disposition_map = {0:'NA', 1:'Discharged home', 2:'Released to police/Jail',3:'Referred to detox/Treatment', 4:'ICU/Critical care', 5:'Surgery', 6:'Chemical dependency/detox, Psychiatric unit', 7:'Other inpatient unit', 8:'Transferred', 9:'Left agaist medical advice', 10:'Died',11:'Other'}

In [21]:
df_grouped = demographic_df.groupby(['AGECAT', 'ALCOHOL']).agg(
        Count=pd.NamedAgg(column='ALCOHOL', aggfunc="count")
    )

index_0 = df_grouped.index.get_level_values(0).map(agecat_map)
index_1 = df_grouped.index.get_level_values(1).map(alcohol_map)
df_grouped_mapped = df_grouped.set_index([index_0, index_1]).reset_index()


In [ ]:
value = 'IL-IN-WI'
metro_map

# Drug dataset

In [3]:
drug_df = df_raw.loc[:, ['DRUGID_1', 'DRUGID_2', 'DRUGID_3']]

In [4]:
route_df = df_raw.loc[:, ['ROUTE_1', 'ROUTE_2', 'ROUTE_3']]

In [10]:
route_map = {1:'Oral', 2:'Injected', 3:'Inhaled/Sniffed/Snorted', 4:'Smoked', 5:'Other', 6:'Transdermal', 98:'Multiple routes'}
series_oral = route_df.eq(1).any(axis=1).astype(int).rename('Oral')
series_injected = route_df.eq(2).any(axis=1).astype(int).rename('Injected')
series_inhaled = route_df.eq(3).any(axis=1).astype(int).rename('Inhaled/Sniffed/Snorted')
series_smoked = route_df.eq(4).any(axis=1).astype(int).rename('Smoked')
series_other = route_df.eq(5).any(axis=1).astype(int).rename('Other')
series_transdermal = route_df.eq(6).any(axis=1).astype(int).rename('Transdermal')
series_multiple = route_df.eq(98).any(axis=1).astype(int).rename('Multiple')
df_routes = pd.concat([series_oral, series_injected, series_inhaled, series_smoked, series_other, series_transdermal, series_multiple], axis=1)

In [11]:
df_routes.to_csv('../Data/routes_data.csv')

In [22]:
drug_map = {1255:'Marijuana', 1254:'Cocaine', 1253:'Heroin', 865:'Alcohol(Ethanol)'}
series_1255 = drug_df.eq(1255).any(axis=1).astype(int).rename('Marijuana')
series_1254 = drug_df.eq(1254).any(axis=1).astype(int).rename('Cocaine')
series_1253 = drug_df.eq(1253).any(axis=1).astype(int).rename('Heroin')
series_865 = drug_df.eq(865).any(axis=1).astype(int).rename('Alcohol(Ethanol)')
df_drug4 = pd.concat([series_1253, series_1254, series_1255, series_865], axis=1)

In [72]:
df_drug4.to_csv('../Data/drug4_data.csv')

In [7]:
pd.read_csv('../Data/routes_data.csv').head()

,Unnamed: 0,Oral,Injected,Inhaled,Smoked,Other,Transderma,Multiple
0,0,1,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0
3,3,0,1,0,0,0,0,0
4,4,1,0,0,0,0,0,0


In [23]:
df_drug4.head()

,Heroin,Cocaine,Marijuana,Alcohol(Ethanol)
0,0,0,1,1
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,1,1,0,1


In [54]:
df_new = pd.concat([demographic_df['SEX'], df_drug4], axis=1)

In [74]:
df_new = pd.concat([demographic_df['SEX'], df_drug4], axis=1)
df_grouped = df_new.groupby('SEX').sum().stack().reset_index()
df_grouped['SEX'] = df_grouped['SEX'].map(sex_map)
df_final = df_grouped.rename(columns={'level_1': 'Drug', 0: 'Count'})

In [75]:
df_final

,SEX,Drug,Count
0,NA,Heroin,3
1,NA,Cocaine,17
2,NA,Marijuana,8
3,NA,Alcohol(Ethanol),18
4,Male,Heroin,13183
5,Male,Cocaine,20630
6,Male,Marijuana,17811
7,Male,Alcohol(Ethanol),27144
8,Female,Heroin,5855
9,Female,Cocaine,9359


In [76]:
px.sunburst(
        df_final,
        path=['Drug', "SEX"],
        values='Count',
        width=600, 
        height=500
    )

Input and output mappings


In [9]:
df_drugmap_raw = pd.read_excel('../Mappings.xlsx', sheet_name='DrugID', usecols='A:C')
df_catmap_raw = pd.read_excel('../Mappings.xlsx', sheet_name='CATID', usecols='A:C')
df_sdled_raw = pd.read_excel('../Mappings.xlsx', sheet_name='SDLED', usecols='A:C')

In [12]:
drug_input_maps = []

def make_dict(group):
    keys = group['VALUE'].values
    vals = group['NAME'].values
    drug_input_maps.append(dict(zip(keys, vals)))
    
df_drugmap_raw.groupby('DRUGID').apply(lambda x: make_dict(x))

""


In [17]:
casetype_map = {0: 'NA', 1:'Suicide Attempt', 2:'Seeking Detox',3:'Alcohol Only(Age<21)', 4:'Adverse Reaction', 5:'Overmedication', 6:'Malicious Poisoning', 7:'Accidental Injestion', 8:'Other'}
df_casetype_count_before = df_raw.groupby('CASETYPE').agg(
        Count_With_Other=pd.NamedAgg(column='CASETYPE', aggfunc="count")
    )
count_after_classification = {'Seeking Detox':8016, 'Adverse Reaction':2128, 'Suicide Attempt':750, 'Overmedication':715, 'Malicious Poisoning': 56, 'Alcohol Only(Age<21)':0, 'Accidental Injestion': 0, 'Other':-83648}
df_casetype_count_after = pd.Series(data=count_after_classification, name='Count_Without_Other')
casetype_index = df_casetype_count_before.index.map(casetype_map)
df_casetype_count_before_mapped = df_casetype_count_before.set_index(casetype_index)

In [27]:
df_casetype_count = pd.concat([df_casetype_count_before_mapped, df_casetype_count_after], axis=1)
df_casetype_count['after_classification'] = df_casetype_count.apply(lambda x: x['Count_With_Other']+x['Count_Without_Other'], axis=1)

In [30]:
df_casetype_count

,Count_With_Other,Count_Without_Other,after_classification
Suicide Attempt,7872,750,8622
Seeking Detox,13529,8016,21545
Alcohol Only(Age<21),7421,0,7421
Adverse Reaction,85777,2128,87905
Overmedication,16810,715,17525
Malicious Poisoning,768,56,824
Accidental Injestion,3125,0,3125
Other,83648,0,83648


In [24]:
df_final = df_casetype_count.unstack()\
    .reset_index()\
    .rename(columns={'level_0':'Classification', 'level_1':'Casetype', 0: 'Count'})

,Classification,Casetype,Count
0,Count_With_Other,Suicide Attempt,7872
1,Count_With_Other,Seeking Detox,13529
2,Count_With_Other,Alcohol Only(Age<21),7421
3,Count_With_Other,Adverse Reaction,85777
4,Count_With_Other,Overmedication,16810
5,Count_With_Other,Malicious Poisoning,768
6,Count_With_Other,Accidental Injestion,3125
7,Count_With_Other,Other,83648
8,Count_Without_Other,Suicide Attempt,750
9,Count_Without_Other,Seeking Detox,8016
